# AI News Digest Agent

In [55]:
from dotenv import load_dotenv
load_dotenv()

True

## Define Web Search Tool

In [56]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for today's most important AI news for an AI Engineer"""

    return tavily_client.search(
        query=query,
        topics='news',
        search_depth='advanced',
        max_results=10,
        time_range='day',
        include_raw_content=True
    )

In [57]:
web_search.invoke("Please search for today's most important AI news for an AI Engineer")

{'query': "Please search for today's most important AI news for an AI Engineer",
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://aijungle.substack.com/p/ai-stars-of-the-week-newsletter-december-1ef',
   'title': 'AI Stars of the Week: Newsletter (December 23, 2025) inkl ...',
   'content': 'This week reveals the maturation paradox: major enterprise vendors launch production-ready AI automation platforms while adoption data shows only 5-18% of',
   'score': 0.98554,
   'raw_content': '🌟 AI Stars of the Week: Newsletter (December 23, 2025) inkl. ⚙️ Automation News\n===============\n\n[![Image 1: The Artificial Intelligence (AI) Jungle ](https://substackcdn.com/image/fetch/$s_!TzwF!,w_80,h_80,c_fill,f_auto,q_auto:good,fl_progressive:steep,g_auto/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F123d156e-84e0-4168-b596-ef23ada8318f_768x768.png)](https://aijungle.substack.com/)\n\n[The Artificial Intelligence (AI) Jungle](http

## Choose the Model

In [58]:
from langchain.chat_models import init_chat_model

model = init_chat_model(model="deepseek-chat")

In [59]:
response = model.invoke("What's an AI agent?")

print(response.content)

An **AI agent** is a software program that can **perceive its environment, make decisions, and take actions** to achieve specific goals autonomously.

Think of it as an intelligent assistant or a digital entity that doesn't just respond to prompts, but proactively works on tasks from start to finish.

---

### **Key Characteristics of an AI Agent:**

1. **Autonomy**  
   It can operate without constant human intervention, making its own choices within defined boundaries.

2. **Perception**  
   It gathers data from its environment via sensors, APIs, user inputs, or real-time data streams.

3. **Decision-Making**  
   It uses reasoning, planning, or machine learning models (like LLMs) to decide what action to take next.

4. **Action**  
   It acts upon the environment through APIs, robotic controls, or digital outputs.

5. **Goal-Oriented**  
   It works toward a specific objective, adjusting its approach as needed.

---

### **Simple Analogy:**
If **ChatGPT** is a conversational brain 

## Defining Response Format

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class AINews(BaseModel):
    title: str
    url: str
    explanation: str = Field(
        description="A detailed explanation (10 sentences minimum) covering: What happened, Why it's significant, How it impacts AI engineers or the industry"
    ) 

class AINewsDigest(BaseModel):
    news_items: List[AINews]

## Create the Agent

In [ ]:
system_prompt = """
You are an AI News Assistant. Your job is to search for and explain the latest AI technology news.

When users ask about AI news:
1. Use the web_search tool to find recent AI news articles from the last 24 hours
2. Choose the 5 most important news items for an AI Engineer
3. For each news item, provide a detailed explanation (10 sentences minimum) for each news item that covers:
   - What happened
   - Why it's significant
   - How it impacts AI engineers or the industry

Be conversational and helpful.
"""

In [76]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="deepseek-chat",
    tools=[web_search],
    system_prompt=system_prompt,
    checkpointer=InMemorySaver(),
    response_format=AINewsDigest,
)

In [ ]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "0"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="Is there new exciting news?")]},
    config
)

In [78]:
pprint(response['structured_response'].news_items)

[AINews(title="3D Chip Breakthrough Claims to Solve AI's Memory Wall Problem", url='https://aijungle.substack.com/p/ai-stars-of-the-week-newsletter-december-1ef', explanation="US researchers have unveiled a monolithic 3D chip architecture that claims to eliminate the memory wall bottleneck constraining AI performance, potentially enabling hardware that's 1,000 times faster than current systems. The memory wall refers to the fundamental mismatch between processor speed and memory access time where modern AI chips spend most of their time waiting for data to move between memory and processing units rather than performing calculations. This data transfer bottleneck has become the primary constraint limiting AI system performance rather than raw computational power. The new 3D architecture addresses this by vertically stacking processing and memory layers with direct connections, dramatically reducing the physical distance data must travel. Instead of data flowing horizontally across a chi

## Check the Conversation History

In [65]:
from pprint import pprint

# Get the current/latest checkpoint
checkpoint = agent.checkpointer.get(config)

pprint(checkpoint['channel_values']['messages'])

[HumanMessage(content='Is there new exciting news?', additional_kwargs={}, response_metadata={}, id='3b0e8429-2012-45bd-9f45-c4bcffd2432f'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 529, 'total_tokens': 572, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 529}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '55ce0d93-7763-44b4-afc4-ba24d3d78d02', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b4afd-4b3e-7c62-a25c-ff49aaf5602f-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'AI news today latest developments artificial intelligence technology'}, 'id': 'call_00_GC9VZdbrkHBmSYgz2C6oThLU', 'type': 'tool_call'}], usage_metadata={'input_tokens': 529, 'output_tokens': 43, 'total_to

## Formatting the Response For the User

In [79]:
for item in response['structured_response'].news_items:
    print(f"Title: {item.title}\nURL: {item.url}\nExplanation: {item.explanation}\n\n")

Title: 3D Chip Breakthrough Claims to Solve AI's Memory Wall Problem
URL: https://aijungle.substack.com/p/ai-stars-of-the-week-newsletter-december-1ef
Explanation: US researchers have unveiled a monolithic 3D chip architecture that claims to eliminate the memory wall bottleneck constraining AI performance, potentially enabling hardware that's 1,000 times faster than current systems. The memory wall refers to the fundamental mismatch between processor speed and memory access time where modern AI chips spend most of their time waiting for data to move between memory and processing units rather than performing calculations. This data transfer bottleneck has become the primary constraint limiting AI system performance rather than raw computational power. The new 3D architecture addresses this by vertically stacking processing and memory layers with direct connections, dramatically reducing the physical distance data must travel. Instead of data flowing horizontally across a chip and throug